60&40 vs All Weather Portfolio
By Licheng Zhong

In [28]:
import numpy as np
import pandas as pd
from scipy.stats import bootstrap

I have gathered the data from yahoo Finance. There's no history of stock split for these ETF. I assume to use close price.
For display purpose, I run the code line by line. However, it may be a better practice to have a dictionary of ETF ticker and its panda frame. Therefore, we could build a pipeline that loops through the dictionary and promotes efficiency.

Another thing is the bond ETF is used as an alternative to bonds. But bonds have maturity, while bonds ETF normally does not. This could make some differences. For example, TIPS and TIP ETF have some differences.

In my humble opinion, if one is determined to hold actual bonds until maturity, then it doesn't make much sense to talk about the daily return or the bond price fluctuation. Since we don't look at everyday price of a bond, then we can't calculate the variance of return from those daily prices. Generally, the only variance of the return should be the default probability. Unlike the case where we use daily price to calculate the empirical variance of the return, there's no empirical way to calculate the default probability of one single bond unless we use the empirical data of a pool of bonds that is regarded to be the same kind as the one we are interested in. In other words, we need to make assumptions. For example, the pool of bonds is a good representative of the bond we are interested in; in other words, the variance of the pool of bonds is indeed the variance of the bond we're interested in. Other assumptions such as iid (so no structural changes, everyday is a fresh new day and so on) or sample size is large enough so that the sample distribution is approximately the population distribution also need to hold. As we can see, this is unlikely. Basically U.S. treasury bonds have never defaulted. Could you say it's default probability is 0? Isn't it possible that its default probability follows a distribution that has a heavy part in 0 but still has other nonzero possible values? However, even if we use price of the bonds, we still face similar assumptions. For example, does the price really fully capture the default probability or other worries such as legal and financial system down, nuclear attack, society collapse, extreme weather and etc.? Is the price really rational?

However, I think this is an over-concern since holding a bond until maturity means we can't liquadate the whole portfolio on any arbitrary day. So it makes more sense to look at everyday price of bonds. 

In [ ]:
# Here assume you download the csv for these tickers. The Cal MIDS application folder may have the csv when working on this

In [29]:
VTI_Price = pd.read_csv("VTI_Price.csv")
VTI_Div = pd.read_csv("VTI.csv")

In [30]:
VTI_Price.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,11/10/2014,104.879997,105.099998,104.690002,105.099998,91.801514,2923500
1,11/11/2014,105.169998,105.239998,104.900002,105.199997,91.888885,1178000
2,11/12/2014,104.809998,105.300003,104.739998,105.180000,91.871422,1089800
3,11/13/2014,105.320000,105.559998,104.730003,105.160004,91.853958,1671100
4,11/14/2014,105.230003,105.330002,104.940002,105.169998,91.862686,2173900


In [31]:
VTI_Div.head()

,Date,Dividends
0,12/22/2014,0.561
1,3/25/2015,0.508
2,6/26/2015,0.468
3,9/25/2015,0.508
4,12/21/2015,0.583


In [32]:
BND_Price = pd.read_csv("BND_Price.csv")
BND_Div = pd.read_csv("BND.csv")

In [33]:
VGLT_Price = pd.read_csv("VGLT_Price.csv")
VGLT_Div = pd.read_csv("VGLT.csv")

In [34]:
VGIT_Price = pd.read_csv("VGIT_Price.csv")
VGIT_Div = pd.read_csv("VGIT.csv")

In [35]:
IAU_Price = pd.read_csv("IAU_Price.csv")
IAU_Div = pd.read_csv("IAU.csv")

In [36]:
PDBC_Price = pd.read_csv("PDBC_Price.csv")
PDBC_Div = pd.read_csv("PDBC.csv")

In [37]:
def combine_price_div(price_pd, div_pd):
    assert len(price_pd) != 0
    
    result = price_pd[["Date", "Close"]].copy()
    
    if (len(div_pd) == 0):
        result["Dividends"] = np.zeros(len(result["Date"]))
        return result[:1737]
    
    result = pd.merge(result, div_pd, on="Date", how="left").fillna(0)
    
    # 1737 is used since the imported data is from Nov 10 2014 to Mar 25 2022. 
    # However the correct date of this analysis is Oct 4 2021. And this notebook is an organized recap of the analysis.
    return result[:1737]
    
def calculate_mean_stddev_sharpe(etf):
    etf_price = etf["Close"]
    forward = etf_price[1:].to_numpy()
    current = etf_price[:len(etf_price)-1].to_numpy()
    dividends = etf["Dividends"][:len(etf_price)-1].to_numpy()
    daily_return = (forward-current+dividends)/current
    expected_daily_return = np.mean(daily_return)
    daily_return_stddev = np.std(daily_return)
    
    # This is technically not the sharpe ratio given risk free rate is not taken care.
    # Since our purpose is to compare portfolio, this is acceptable.
    annualized_sharpe_ratio = (252**0.5)*expected_daily_return/daily_return_stddev
    return (expected_daily_return, daily_return_stddev, annualized_sharpe_ratio)

def single_etf_pipeline(price_pd, div_pd):
    return calculate_mean_stddev_sharpe(combine_price_div(price_pd, div_pd))

# This portfolio is based on 60% stock and 40% bond
def sixty_forty_portfolio_pipeline(stock, bond):
    etf_price = stock["Close"].to_numpy()*0.6 + bond["Close"].to_numpy()*0.4
    etf_div = stock["Dividends"].to_numpy()*0.6 + bond["Dividends"].to_numpy()*0.4
    
    forward = etf_price[1:]
    current = etf_price[:len(etf_price)-1]
    dividends = etf_div[:len(etf_price)-1]
    daily_return = (forward-current+dividends)/current
    expected_daily_return = np.mean(daily_return)
    daily_return_stddev = np.std(daily_return)
    
    # This is technically not the sharpe ratio given risk free rate is not taken care.
    # Since our purpose is to compare portfolio, this is acceptable.
    annualized_sharpe_ratio = (252**0.5)*expected_daily_return/daily_return_stddev
    return (expected_daily_return, daily_return_stddev, annualized_sharpe_ratio)

# This portfolio tries to mimic Ray Dalio's All Weather Portfolio. There's no guarentee that it's really the case.
# The portfolio is made of 30% VTI, 40% VGLT, 15% VGIT, 8% IAU, 7% PDBC
# https://www.optimizedportfolio.com/all-weather-portfolio/
# It's also suggested to use REIT or Utility ETF suhc as VPU to substitute PDBC.
# I feel it's worth
def all_weather_portfolio_pipeline(stock, long_bond, short_bond, gold, commodities):
    etf_price = stock["Close"].to_numpy()*0.3 + long_bond["Close"].to_numpy()*0.4 +\
                short_bond["Close"].to_numpy()*0.15 + gold["Close"].to_numpy()*0.08 + commodities["Close"].to_numpy()*0.07
    etf_div = stock["Dividends"].to_numpy()*0.3 + long_bond["Dividends"].to_numpy()*0.4 +\
                short_bond["Dividends"].to_numpy()*0.15 + gold["Dividends"].to_numpy()*0.08 +\
                commodities["Dividends"].to_numpy()*0.07
    
    forward = etf_price[1:]
    current = etf_price[:len(etf_price)-1]
    dividends = etf_div[:len(etf_price)-1]
    daily_return = (forward-current+dividends)/current
    expected_daily_return = np.mean(daily_return)
    daily_return_stddev = np.std(daily_return)
    
    # This is technically not the sharpe ratio given risk free rate is not taken care.
    # Since our purpose is to compare portfolio, this is acceptable.
    annualized_sharpe_ratio = (252**0.5)*expected_daily_return/daily_return_stddev
    return (expected_daily_return, daily_return_stddev, annualized_sharpe_ratio)

In [38]:
VTI = combine_price_div(VTI_Price, VTI_Div)
VTI_info = calculate_mean_stddev_sharpe(VTI)
VTI_info

(0.0005691740927613655, 0.011364958541000348, 0.795019056188215)

In [39]:
BND = combine_price_div(BND_Price, BND_Div)
BND_info = calculate_mean_stddev_sharpe(BND)
BND_info

(0.00013321079998841498, 0.0030063489424147567, 0.7033966890767309)

In [40]:
VGLT = combine_price_div(VGLT_Price, VGLT_Div)
VGLT_info = calculate_mean_stddev_sharpe(VGLT)
VGLT_info

(0.0002350899552468904, 0.008241642496442925, 0.45281475694755663)

In [41]:
VGIT = combine_price_div(VGIT_Price, VGIT_Div)
VGIT_info = calculate_mean_stddev_sharpe(VGIT)
VGIT_info

(0.00010522190538599393, 0.0023136771066901538, 0.7219442851106269)

In [42]:
IAU = combine_price_div(IAU_Price, IAU_Div)
IAU_info = calculate_mean_stddev_sharpe(IAU)
IAU_info

(0.0005640497881070041, 0.028386838313119493, 0.3154283227872054)

In [43]:
PDBC = combine_price_div(PDBC_Price, PDBC_Div)
PDBC_info = calculate_mean_stddev_sharpe(PDBC)
PDBC_info

(5.0510876225123846e-05, 0.01115490498651671, 0.07188185851363682)

In [44]:
sixty_forty_portfolio_pipeline(VTI, BND)

(0.0004363054383954725, 0.008112273579222454, 0.853784582869116)

In [45]:
all_weather_portfolio_pipeline(VTI, VGLT, VGIT, IAU, PDBC)

(0.00035252021351786983, 0.005182691781099125, 1.079764172540325)

In [26]:
sixty_forty_portfolio_pipeline(VTI, BND)[1]*(252**0.5)

0.1287783507476547

In [27]:
all_weather_portfolio_pipeline(VTI, VGLT, VGIT, IAU, PDBC)[1]*(252**0.5)

0.08139673571643213

In [21]:
def sixty_forty_portfolio_bootstrap_pipeline(stock, bond):
    etf_price = stock["Close"].to_numpy()*0.6 + bond["Close"].to_numpy()*0.4
    etf_div = stock["Dividends"].to_numpy()*0.6 + bond["Dividends"].to_numpy()*0.4
    
    forward = etf_price[1:]
    current = etf_price[:len(etf_price)-1]
    dividends = etf_div[:len(etf_price)-1]
    daily_return = (forward-current+dividends)/current
    
    daily_return = (daily_return,)
   
    expected_daily_return = bootstrap(daily_return, np.mean)
    daily_return_stddev = bootstrap(daily_return, np.std)
    
    return (expected_daily_return, daily_return_stddev)

def all_weather_portfolio_bootstrap_pipeline(stock, long_bond, short_bond, gold, commodities):
    etf_price = stock["Close"].to_numpy()*0.3 + long_bond["Close"].to_numpy()*0.4 +\
                short_bond["Close"].to_numpy()*0.15 + gold["Close"].to_numpy()*0.08 + commodities["Close"].to_numpy()*0.07
    etf_div = stock["Dividends"].to_numpy()*0.3 + long_bond["Dividends"].to_numpy()*0.4 +\
                short_bond["Dividends"].to_numpy()*0.15 + gold["Dividends"].to_numpy()*0.08 +\
                commodities["Dividends"].to_numpy()*0.07
    
    forward = etf_price[1:]
    current = etf_price[:len(etf_price)-1]
    dividends = etf_div[:len(etf_price)-1]
    daily_return = (forward-current+dividends)/current
    daily_return = (daily_return,)
   
    expected_daily_return = bootstrap(daily_return, np.mean)
    daily_return_stddev = bootstrap(daily_return, np.std)
   
    return (expected_daily_return, daily_return_stddev)

In [22]:
sixty_forty_bootstrap_res = sixty_forty_portfolio_bootstrap_pipeline(VTI, BND)
sixty_forty_return_res, sixty_forty_stddev_res = sixty_forty_bootstrap_res[0], sixty_forty_bootstrap_res[1]
all_weather_bootstrap_res = all_weather_portfolio_bootstrap_pipeline(VTI, VGLT, VGIT, IAU, PDBC)
all_weather_return_res, all_weather_stddev_res = all_weather_bootstrap_res[0], all_weather_bootstrap_res[1]

In [75]:
sixty_forty_return_res.confidence_interval

ConfidenceInterval(low=4.4254618552752896e-05, high=0.0008091924942903599)

In [76]:
sixty_forty_stddev_res.confidence_interval

ConfidenceInterval(low=0.007333056162925891, high=0.009297797571527813)

In [77]:
all_weather_return_res.confidence_interval

ConfidenceInterval(low=0.00010046273198481245, high=0.0005945257271046198)

In [78]:
all_weather_stddev_res.confidence_interval

ConfidenceInterval(low=0.004811213346973598, high=0.005778461799732432)

The following is not the correct way to calculate sharpe ratio confidence interval. But it may give us some ideas.

In [81]:
(252**0.5)*sixty_forty_return_res.confidence_interval.high/sixty_forty_stddev_res.confidence_interval.low

1.7517297468656923

In [82]:
(252**0.5)*all_weather_return_res.confidence_interval.high/all_weather_stddev_res.confidence_interval.low

1.9616264445285372

In [83]:
(252**0.5)*sixty_forty_return_res.confidence_interval.low/sixty_forty_stddev_res.confidence_interval.high

0.07555770976246559

In [84]:
(252**0.5)*all_weather_return_res.confidence_interval.low/all_weather_stddev_res.confidence_interval.high

0.27598978490184617

In [96]:
def std_diff(a, b):
    return np.std(a)-np.std(b)

def test_risk(stock, bond, long_bond, short_bond, gold, commodities):
    etf_price = stock["Close"].to_numpy()*0.6 + bond["Close"].to_numpy()*0.4
    etf_div = stock["Dividends"].to_numpy()*0.6 + bond["Dividends"].to_numpy()*0.4
    
    forward = etf_price[1:]
    current = etf_price[:len(etf_price)-1]
    dividends = etf_div[:len(etf_price)-1]
    daily_return_1 = (forward-current+dividends)/current
    

    etf_price = stock["Close"].to_numpy()*0.3 + long_bond["Close"].to_numpy()*0.4 +\
                short_bond["Close"].to_numpy()*0.15 + gold["Close"].to_numpy()*0.08 + commodities["Close"].to_numpy()*0.07
    etf_div = stock["Dividends"].to_numpy()*0.3 + long_bond["Dividends"].to_numpy()*0.4 +\
                short_bond["Dividends"].to_numpy()*0.15 + gold["Dividends"].to_numpy()*0.08 +\
                commodities["Dividends"].to_numpy()*0.07
    
    forward = etf_price[1:]
    current = etf_price[:len(etf_price)-1]
    dividends = etf_div[:len(etf_price)-1]
    daily_return_2 = (forward-current+dividends)/current
    daily_return = (daily_return_1, daily_return_2,)
   
    daily_return_stddev = bootstrap(daily_return, std_diff, method="percentile", vectorized=False)
    #daily_return_stddev = std_diff(daily_return_1, daily_return_2)
   
    return daily_return_stddev

In [97]:
test_risk(VTI, BND, VGLT, VGIT, IAU, PDBC)

BootstrapResult(confidence_interval=ConfidenceInterval(low=0.0018940789034736338, high=0.004001169478868422), standard_error=0.0005363582393951279)

It seems that all weather portfolio performs better. Now let's see my gain. Assume I bought $1000 for each portfolio at Oct 4 2021. See how much this 1000 becomes at Mar 25 2022.

In [48]:
def combine_price_div(price_pd, div_pd):
    assert len(price_pd) != 0
    
    result = price_pd[["Date", "Close"]].copy()
    
    if (len(div_pd) == 0):
        result["Dividends"] = np.zeros(len(result["Date"]))
        return result[1737:]
    
    result = pd.merge(result, div_pd, on="Date", how="left").fillna(0)
    
    # 1737 is used since the imported data is from Nov 10 2014 to Mar 25 2022. 
    # However the correct date of this analysis is Oct 4 2021. And this notebook is an organized recap of the analysis.
    return result[1737:]

# This portfolio is based on 60% stock and 40% bond
def sixty_forty_portfolio_pipeline(stock, bond):
    etf_price = stock["Close"].to_numpy()*0.6 + bond["Close"].to_numpy()*0.4
    etf_div = stock["Dividends"].to_numpy()*0.6 + bond["Dividends"].to_numpy()*0.4
    
    forward = etf_price[1:]
    current = etf_price[:len(etf_price)-1]
    dividends = etf_div[:len(etf_price)-1]
    
    return 1000*etf_price[-1]/etf_price[0]+(1000/etf_price[0])*sum(dividends)

# This portfolio tries to mimic Ray Dalio's All Weather Portfolio. There's no guarentee that it's really the case.
# The portfolio is made of 30% VTI, 40% VGLT, 15% VGIT, 8% IAU, 7% PDBC
# https://www.optimizedportfolio.com/all-weather-portfolio/
# It's also suggested to use REIT or Utility ETF suhc as VPU to substitute PDBC.
# I feel it's worth
def all_weather_portfolio_pipeline(stock, long_bond, short_bond, gold, commodities):
    etf_price = stock["Close"].to_numpy()*0.3 + long_bond["Close"].to_numpy()*0.4 +\
                short_bond["Close"].to_numpy()*0.15 + gold["Close"].to_numpy()*0.08 + commodities["Close"].to_numpy()*0.07
    etf_div = stock["Dividends"].to_numpy()*0.3 + long_bond["Dividends"].to_numpy()*0.4 +\
                short_bond["Dividends"].to_numpy()*0.15 + gold["Dividends"].to_numpy()*0.08 +\
                commodities["Dividends"].to_numpy()*0.07
    
    forward = etf_price[1:]
    current = etf_price[:len(etf_price)-1]
    dividends = etf_div[:len(etf_price)-1]
    return 1000*etf_price[-1]/etf_price[0]+(1000/etf_price[0])*sum(dividends)

In [49]:
VTI = combine_price_div(VTI_Price, VTI_Div)
BND = combine_price_div(BND_Price, BND_Div)
VGLT = combine_price_div(VGLT_Price, VGLT_Div)
VGIT = combine_price_div(VGIT_Price, VGIT_Div)
IAU = combine_price_div(IAU_Price, IAU_Div)
PDBC = combine_price_div(PDBC_Price, PDBC_Div)

In [50]:
sixty_forty_portfolio_pipeline(VTI, BND)

1007.0575111544539

In [51]:
all_weather_portfolio_pipeline(VTI, VGLT, VGIT, IAU, PDBC)

984.8812871079547

In [54]:
sum(BND["Dividends"])

0.806

In [55]:
sum(VGLT["Dividends"])

0.661

In [56]:
sum(VGIT["Dividends"])

0.6729999999999998